In [ ]:
import os
path_with_mask = r'C:\\Users\\Arshit\\Covid19 Mask Detection\\train\\with_mask'
path_without_mask = r'C:\\Users\\Arshit\\Covid19 Mask Detection\\train\\without_mask'

In [ ]:
li_with_mask = os.listdir(path_with_mask)
li_without_mask = os.listdir(path_without_mask)

print(len(li_with_mask))
print(len(li_without_mask))

In [ ]:
import cv2

In [ ]:
X_train = []
for i in range(341):
    
    img = cv2.imread(path_with_mask+'\\'+li_with_mask[i])
    img = cv2.resize(img,(64,64))
    X_train.append(img)

In [ ]:
for i in range(275):
    img = cv2.imread(path_without_mask+'\\'+li_without_mask[i])
    img = cv2.resize(img,(64,64))
    X_train.append(img)

In [ ]:
import numpy as np

In [ ]:
X_train[0].shape

In [ ]:
X_train = np.reshape(X_train,(len(X_train),)+(64,64,3))

In [ ]:
Y_train = []
for i in range(len(li_with_mask)):
    Y_train.append(1)
    
for j in range(len(li_without_mask)):
    Y_train.append(0)

In [ ]:
Y_train = np.reshape(Y_train,(len(Y_train),)+(1,))
Y_train = Y_train.T

In [ ]:
print(X_train.shape)
print(Y_train.shape)

In [ ]:
X_train_flatten = X_train.reshape(X_train.shape[0], -1).T
X_train = X_train_flatten/255.

In [ ]:
X_train.shape

In [ ]:
layers_dims = [12288, 20, 7, 5, 1] #  4-layer model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

def sigmoid(Z):
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, cache

def relu(Z):
    
    A = np.maximum(0,Z)
    assert(A.shape == Z.shape)
    cache = Z 
    return A, cache

def relu_backward(dA, cache):
    
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape)
    return dZ

def sigmoid_backward(dA, cache):
    
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    assert (dZ.shape == Z.shape)
    return dZ


def initialize_parameters_deep(layer_dims):
    
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)         

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1]) #*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters



def linear_forward(A, W, b):
    
    Z = W.dot(A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

def linear_activation_forward(A_prev, W, b, activation):

    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache


def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2                  
    

    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
    

    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

def compute_cost(AL, Y):

    m = Y.shape[1]


    cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    
    cost = np.squeeze(cost)      
    assert(cost.shape == ())
    
    return cost

def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1./m * np.dot(dZ,A_prev.T)
    db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T,dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

def L_model_backward(AL, Y, caches):
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2 
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

In [ ]:
def model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):

    np.random.seed(1)
    costs = []                         
    
    parameters = initialize_parameters_deep(layers_dims)
    
    for i in range(0, num_iterations):

        AL, caches = L_model_forward(X,parameters)  
        cost = compute_cost(AL,Y)
        grads = L_model_backward(AL,Y,caches)
        parameters = update_parameters(parameters,grads,learning_rate)
                
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [ ]:
parameters = model(X_train, Y_train, layers_dims, num_iterations = 2500, print_cost = True)

In [ ]:
def predict(X, y, parameters):
    
    m = X.shape[1]
    n = len(parameters) // 2 
    p = np.zeros((1,m))

    probas, caches = L_model_forward(X, parameters)
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    
    print("Accuracy: "  + str(np.sum((p == y)/m)))
        
    return p

In [ ]:
pred = predict(X_train, Y_train, parameters)

In [ ]:
def predict_random(X,parameters):
    
    m = X.shape[1]
    n = len(parameters) // 2 
    p = np.zeros((1,m))
    
    probas, caches = L_model_forward(X, parameters)
    
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
            a="Mask detected"
        else:
            p[0,i] = 0
            a="Mask not detected"
        
    return p,probas,a

In [ ]:
random_image = cv2.imread('79-with-mask.jpg')
random_image = cv2.resize(random_image,(64,64))
random_img = np.reshape(random_image,(1,)+(64,64,3))
random_img_flatten = random_img.reshape(random_img.shape[0], -1).T
random_img_flatten = random_img_flatten/255.
Y = 1
Y = np.reshape(Y,(1,)+(1,))
pred,_,a = predict_random(random_img_flatten,parameters)

In [ ]:
a

In [ ]:
random_image = cv2.imread('79-with-mask.jpg')
random_image_check = cv2.resize(random_image,(300,300))
cv2.imshow('img',random_image_check)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
face_classifier = cv2.CascadeClassifier(r'C:\Users\Arshit\haarcascades\haarcascade_frontalface_default.xml')

In [ ]:
from keras.preprocessing import image

In [ ]:
# static image predictions

random_img = cv2.imread('79-with-mask.jpg')
faces=face_classifier.detectMultiScale(random_img, scaleFactor=1.3, minNeighbors=3)
for x,y,w,h in faces:
    
    img= cv2.rectangle(random_img, (x,y), (x+w,y+h), (0,255,0),2)
    img = cv2.resize(img, (64,64))
    img = np.reshape(img,(1,)+(64,64,3))
    img_flatten = img.reshape(img.shape[0], -1).T
    img_flatten = img_flatten/255.
    pred,_,a = predict_random(img_flatten,parameters)
    print(a)
    if a=='Mask detected':
        cv2.putText(random_img, a, (int(x),int(y)) ,cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    else:
        cv2.putText(random_img, a, (int(x),int(y)) ,cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
cv2.imshow('img', random_img)
cv2.waitKey(0)
cv2.destroyAllWindows()    

In [ ]:
# realtime predictions

cap = cv2.VideoCapture(0)
while True:
    ret, random_img = cap.read()
    faces=face_classifier.detectMultiScale(random_img, scaleFactor=1.3, minNeighbors=3)
    for x,y,w,h in faces:
    
        img= cv2.rectangle(random_img, (x,y), (x+w,y+h), (0,255,0),2)
        img = cv2.resize(img, (64,64))
        img = np.reshape(img,(1,)+(64,64,3))
        img_flatten = img.reshape(img.shape[0], -1).T
        img_flatten = img_flatten/255.
        pred,_,a = predict_random(img_flatten,parameters)
        if a=='Mask detected':
            cv2.putText(random_img, a, (int(x),int(y)) ,cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        else:
            cv2.putText(random_img, a, (int(x),int(y)) ,cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        
    cv2.imshow('img', random_img)
    if cv2.waitKey(1)==27:
        break
cap.release()        
cv2.destroyAllWindows()    